In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from argparse import Namespace
import matplotlib.pyplot as plt
import av2.rendering.vector as vector_plotting_utils
from shapely.geometry import Polygon

from graph_creator.MapGraph import MapGraph
from graph_creator.ActorGraph import ActorGraph

from av2.map.map_api import ArgoverseStaticMap
from av2.datasets.motion_forecasting import scenario_serialization

from av2.datasets.motion_forecasting.viz.scenario_visualization import (
    visualize_scenario
)
from av2.map.map_api import ArgoverseStaticMap
import os
import pickle
from tqdm import tqdm
import networkx as nx
os.getcwd()


from graph_creator.create_graph import get_scenario_data, plot_scene_at_timestep

In [2]:
from pathlib import Path
# repo_root = Path("/Users/marius/code/graph_coverage")
repo_root = Path("/home/tmuehlen/repos/graph_coverage")
dataroot = repo_root / "argoverse_data" / "train"
print(repo_root)


/home/tmuehlen/repos/graph_coverage


In [3]:
scenario_folders = [f for f in dataroot.iterdir() if f.is_dir()]

# Print number of scenarios found
print(f"Found {len(scenario_folders)} scenarios")

Found 199908 scenarios


In [4]:
# @ Thomas run here for data creation for one scenario:

i = 1001
scenario = scenario_folders[i]
log_id = scenario.name
# check if file exists:
timestamp = 1.0
print(i, log_id)
scenario, map = get_scenario_data(dataroot, log_id)
G_map = MapGraph.create_from_argoverse_map(map)
actor_graph = ActorGraph.from_argoverse_scenario(
                                scenario, 
                                G_map, 
                                delta_timestep_s=1.0,
                                max_distance_lead_veh_m=50,
                                max_distance_opposite_forward_m=100,
                                max_distance_opposite_backward_m=10,
                                max_distance_neighbor_forward_m=50,
                                max_distance_neighbor_backward_m=20,
                                max_node_distance_leading=10,
                                max_node_distance_neighbor=6,
                                max_node_distance_opposite=6,
)

timestamps = list(actor_graph.actor_graphs.keys())
timestamp = timestamps[2]
actor_graph.actor_graphs[timestamp]
# check if graph is connected:
# nx.is_connected(actor_graph.actor_graphs[timestamp]) # not implemented for directed graphs... 
#dir(actor_graph)

actor_graph.actor_components[timestamp]

keys = list(actor_graph.actor_graphs.keys())
for  key in keys:
    actor_graph.actor_components[key] = [actor_graph.actor_components[key][i] for i in range(len(actor_graph.actor_components[key])) if actor_graph.actor_components[key][i].size() > 1]

for timestamp in timestamps:
    for component_idx in range(len(actor_graph.actor_components[timestamp])):
        # save_path = f'/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse_components/graph_{log_id}_{timestamp}_{component_idx}.pkl'
        save_path = f'/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse_components_nx/graph_{log_id}_{str(timestamp).replace(".", "_")}_{component_idx}.pkl'
        # with open(save_path, "wb") as file:
        #     pickle.dump(actor_graph.actor_components[timestamp][component_idx], file)

1001 1a2add4b-20c3-405a-8149-d4f9c06d3fa4


In [5]:
# @ Thomas run here for data creation
actor_graphs = []

for i, scenario in enumerate(scenario_folders):
    if i >= 1000:
        break
    log_id = scenario.name
    # check if file exists:
    # if (repo_root / "actor_graphs" / "argoverse" / f"{log_id}_map_graph.pkl").exists():
    #     continue
    timestamp = 1.0
    print(i, log_id)
    scenario, map = get_scenario_data(dataroot, log_id)
    G_map = MapGraph.create_from_argoverse_map(map)
    actor_graph = ActorGraph.from_argoverse_scenario(
                                    scenario, 
                                    G_map, 
                                    delta_timestep_s=1.0,
                                    max_distance_lead_veh_m=50,
                                    max_distance_opposite_forward_m=100,
                                    max_distance_opposite_backward_m=10,
                                    max_distance_neighbor_forward_m=50,
                                    max_distance_neighbor_backward_m=20,
                                    max_node_distance_leading=10,
                                    max_node_distance_neighbor=6,
                                    max_node_distance_opposite=6,
    )
    # actor_graphs.append(actor_graph)
    #actor_graph.visualize_actor_graph(t_idx=timestamp, comp_idx=0,  save_path=(repo_root / "plots" / f"{log_id}_actor_graph.png"))
    #visualize_scenario(scenario, map, save_path=(repo_root / f"{log_id}_video_.mp4") )
    #plot_scene_at_timestep(scenario, map, timestamp, save_path=(repo_root / "plots" / f"{log_id}_scene_at_{timestamp}.png"), lane_label=True, actor_graph=actor_graph)
    #G_map.visualize_graph(save_path=(repo_root / "plots" / f"{log_id}_map_graph.png"))
    with open(repo_root / "actor_graphs" / "argoverse" / f"{log_id}_map_graph.pkl", "wb") as f:
        pickle.dump(G_map, f)
    with open(repo_root / "actor_graphs" / "argoverse" / f"{log_id}_actor_graph.pkl", "wb") as f:
        pickle.dump(actor_graph, f)
    
    keys = list(actor_graph.actor_graphs.keys())
    for  key in keys:
        actor_graph.actor_components[key] = [actor_graph.actor_components[key][i] for i in range(len(actor_graph.actor_components[key])) if actor_graph.actor_components[key][i].size() > 1]

    for timestamp in timestamps:
        for component_idx in range(len(actor_graph.actor_components[timestamp])):
            # save_path = f'/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse_components/graph_{log_id}_{timestamp}_{component_idx}.pkl'
            save_path = f'/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse_components_nx/graph_{log_id}_{str(timestamp).replace(".", "_")}_{component_idx}.pkl'
            with open(save_path, "wb") as file:
                pickle.dump(actor_graph.actor_components[timestamp][component_idx], file)
        save_path = f'/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse_nx/graph_{log_id}_{str(timestamp).replace(".", "_")}.pkl'
        with open(save_path, "wb") as file:
            pickle.dump(actor_graph.actor_graphs[timestamp], file)

    

0 703648e4-8d09-4698-8b82-85ee427182f5
1 a14dcb01-6232-4160-8687-a9cab584bed4
2 8024efab-39c1-4499-8e6e-e440b68929cf
3 22d6a673-c400-4d8f-8b73-d75b817f7410
4 b7ff5ba5-836b-48fa-ad0a-c155fcb19abd
5 0c9f192d-ed7b-4910-95a8-d282a45cf8e2
6 94e791e6-b38b-4666-ad70-86686fb9344b
7 5d93ceaa-af78-4614-935e-fbe04d6cda84
8 830bc57b-b685-4d5c-82ad-ee9db3a52d44
9 a4983be7-4efd-4102-b935-5b969d5aa77f
10 d5a14cc3-4263-4ada-a9ad-7fba5afa86c3
11 03853ba7-4fbe-4968-b4c5-1a32716bc461
12 3d5378ff-8743-4c4b-99b5-85cb159ffc45
13 366e0dbd-6160-4c2b-96ca-01a39b747b32
14 0572aa4a-dd27-4eed-b2db-defe01834922
15 43fcf330-847b-4008-b748-d8791a8e38e9
16 8761a8a5-27c0-47bb-9126-624ed2b9c987
17 41b87a9d-b1db-4311-9abc-52da475b5717
18 d76ea7ca-4474-46c0-ab06-b5b6066f723c
19 41f51907-ac3c-4bbc-90ed-bb22bf273846
20 6030a6bf-7d85-4067-9cf6-255ba3715d9a
21 9d931117-d987-4099-8336-c94b2dba1af2
22 fb069328-d32a-4d40-ae8d-d434a8111bfe
23 84cc51c1-e8b9-4719-bf18-a5d7cf71b2ce
24 8fe9470d-2808-4afb-9499-e10c6ad4bbf2
25 5013d3f

In [ ]:
# # and now split the graphs into networkx data:

# argoverse_scns = os.listdir("/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse")
# argoverse_scns = [scn.split("_")[0] for scn in argoverse_scns if scn.endswith("_actor_graph.pkl")]
# print(argoverse_scns[:3])

# for scn in tqdm(argoverse_scns):
#     with open(f"/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse/{scn}_actor_graph.pkl", "rb") as f:
#         actor_graph = pickle.load(f)
#     timestamps = list(actor_graph.actor_graphs.keys())
#     for timestamp in timestamps:
#         # actor_graph.visualize_actor_graph(timestamp, 0)
#         save_path = f'/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse_nx/graph_{scn}_{str(timestamp).replace(".", "_")}.pkl'
#         with open(save_path, "wb") as file:
#             pickle.dump(actor_graph.actor_graphs[timestamp], file)
        


In [ ]:
# actor_graphs = []
# import pickle

# for i, scenario in enumerate(scenario_folders):
#     if i >= 5:
#         break
# #if True: 
#     #log_id = "d25d1aaa-8bb2-4c6b-98a5-fa5aa2ddd2eb"
#     log_id = scenario.name
#     print(log_id)
#     timestamp = 1.0
#     scenario, map = get_scenario_data(dataroot, log_id)
#     G_map = MapGraph.create_from_argoverse_map(map)
#     actor_graph = ActorGraph.from_argoverse_scenario(
#                                     scenario, 
#                                     G_map, 
#                                     delta_timestep_s=1.0,
#                                     max_distance_lead_veh_m=50,
#                                     max_distance_opposite_forward_m=100,
#                                     max_distance_opposite_backward_m=10,
#                                     max_distance_neighbor_forward_m=50,
#                                     max_distance_neighbor_backward_m=20,
#                                     max_node_distance_leading=10,
#                                     max_node_distance_neighbor=6,
#                                     max_node_distance_opposite=6,
#     )
#     actor_graphs.append(actor_graph)
#     actor_graph.visualize_actor_graph(t_idx=timestamp, comp_idx=0,  save_path=(repo_root / "plots" / f"{log_id}_actor_graph.png"))
#     #visualize_scenario(scenario, map, save_path=(repo_root / f"{log_id}_video_.mp4") )
#     plot_scene_at_timestep(scenario, map, timestamp, save_path=(repo_root / "plots" / f"{log_id}_scene_at_{timestamp}.png"), lane_label=True, actor_graph=actor_graph)
#     G_map.visualize_graph(save_path=(repo_root / "plots" / f"{log_id}_map_graph.png"))
#     #with open(repo_root / "data" / f"{log_id}_map_graph.pkl", "wb") as f:
#     #    pickle.dump(G_map, f)
#     #with open(repo_root / "data" / f"{log_id}_actor_graph.pkl", "wb") as f:
#     #    pickle.dump(actor_graph, f)


In [ ]:
import pickle
# Save actor graphs and map data to pickle files
output_dir = repo_root / "data"
output_dir.mkdir(exist_ok=True)

# Save actor graphs
actor_graphs_file = output_dir / "actor_graph_argoverse.pkl"
with open(actor_graphs_file, "wb") as f:
    pickle.dump(actor_graphs, f)

# Save map data for each scenario
map_file = output_dir / f"map_argoverse.pkl"
with open(map_file, "wb") as f:
    pickle.dump(G_map, f)



In [ ]:
from graph_creator.SubgraphExtractor import SubgraphExtractor
import os

# Example usage of the SubgraphExtractor
def analyze_actor_graphs(actor_graphs, strategy='frequency', output_dir=None):
    """
    Analyze actor graphs to find common subgraphs using the specified strategy.
    
    Args:
        actor_graphs: List of ActorGraph objects
        strategy: Strategy for selecting subgraphs
        output_dir: Directory to save visualizations
    
    Returns:
        SubgraphExtractor instance
    """
    
    # Initialize extractor with specified strategy
    extractor = SubgraphExtractor(
        min_subgraph_size=3,
        max_subgraph_size=5, 
        # Custom matchers can be defined here if needed
    )
    
    # Extract subgraphs
    subgraph_library = extractor.extract_subgraphs(actor_graphs[:10])
    
    # Print properties
    extractor.print_subgraph_properties()
    
    # Visualize subgraphs
    if output_dir is not None:
        strategy_dir = os.path.join(output_dir, f"strategy_{strategy}")
        extractor.visualize_subgraphs(output_dir=strategy_dir)
    else:
        extractor.visualize_subgraphs()
    
    return extractor

# Create visualization output directory
output_dir = repo_root /  "plots" / "subgraph_analysis"
os.makedirs(output_dir, exist_ok=True)

# Analyze with different strategies
strategies = ['frequency', 'density', 'coverage', 'complexity', 'size']
extractors = {}

for strategy in strategies:
    extractors[strategy] = analyze_actor_graphs(actor_graphs, strategy, output_dir)

# Example of how to access the subgraphs for a specific actor graph and timestamp
print("\nExample of subgraph decomposition:")
print("=================================")
example_graph_idx = 0  # First graph in the list
example_timestamp = list(actor_graphs[example_graph_idx].actor_graphs.keys())[0]
print(f"Graph {example_graph_idx}, Timestamp {example_timestamp}:")

for sg_info in actor_graphs[example_graph_idx].actor_subgraphs[example_timestamp]:
    sg_id = sg_info['subgraph_id']
    nodes = sg_info['nodes']
    print(f"  Subgraph ID: {sg_id}, Nodes: {nodes}")
    # Additional information can be accessed from sg_info['node_mapping']